In [ ]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pylab as plt

# Fix the random seed to facilitate grading
np.random.seed(1)

# HW2.2 - Applying constrained optimization

## 2.a Forward Kinematics with IPOPT (26 pts)

The problem we study in this homework is the simplified model of our pick-and-place compliant robotics platform that we are using in the practicals of the class.

In lectures, we derived a simple model for this system. The optimization problem that we can solve to calculate the forward kinematics calculates, given the lengths $l_1$ and $l_2$, the end-effector position $p$ at equilibrium. We can do this by minimizing the total potential energy. The resulting optimization formulation is:

\begin{aligned}
\min_{\delta_1,\delta_2,\theta_1,\theta_2,p}\quad &
\frac{1}{2}k_1 \delta_1^2 + \frac{1}{2}k_2 \delta_2^2 - mg\,p_y\\
\text{s.t.}\quad 
& (\ell_1+\delta_1)\cos\theta_1 = p_x,\\
& (\ell_1+\delta_1)\sin\theta_1 = p_y,\\
& b+(\ell_2+\delta_2)\cos\theta_2 = p_x,\\
& (\ell_2+\delta_2)\sin\theta_2 = p_y,\\
& \delta_1\ge 0,\ \delta_2\ge 0,\ \ell_1\ge 0,\ \ell_2\ge 0,\\
& 0\le \theta_1\le \pi,\ 0\le \theta_2\le \pi.
\end{aligned}
**where:**

- $p = (p_x, p_y)$ is the end-effector position, where the x-axis points to the right an the y-axis points downward.  
- $\ell_1, \ell_2$ are the controllable arm lengths governed by the corresponding motor angles  
- $\delta_1, \delta_2$ are the spring extensions  
- $\theta_1, \theta_2$ are the arm orientation angles measured from the horizontal axis, clockwise
- $k_1, k_2$ are the spring stiffness coefficients  
- $m$ is the end-effector mass and $g$ is the gravitational acceleration  
- $b$ is the distance between the two arm bases

**2.a.1** Using the autodifferentiation library __casadi__ and the general-purpose solver __IPOPT__, solve this problem numerically. (10 pts)

. 

In [ ]:
import casadi as ca

k1 = 100.0
k2 = 120.0
m  = 5.0
grav  = 9.81
b = 2.0

def solve_forward_kinematics(l, d0=None, t0=None, p0=None, verbose=True):
    """
    returns:
    - f, the optimal cost
    - theta, the vector of two angles at equilibrium
    - delta, the vector of two spring extensions at equilibrium
    - p, the end-effector position at equilibrium
    - lambda, the dual variables of the equality constraints
    - mus, the dual variables of the inequality constraints
    """
    ### ============= 2.a.1 Fill in below ==============
    opti = ca.Opti()
    
    # ----------------------------
    # Decision variables
    delta = opti.variable(2)
    theta = opti.variable(2)
    p = opti.variable(2)

    NotImplementedError

    return f, theta, delta, p, lambdas, mus
    
    ### ===========================

f, theta, delta, p, lambdas, mus = solve_forward_kinematics([0.5, 1.0])

**2.a.2** Verify that your implementation works using the below plotting function: solve forward kinematics over a selection of lengths and visualize the resulting equilibrium end-effector position $p_x, p_y$. Comment on why you believe this shows that the function works as expected, commenting on the trend and a specific example (4 pts)

In [ ]:
def draw_setup(p, ax=None, color="k", label=None):
    if ax is None:
        fig, ax = plt.subplots()
    else:
        fig = plt.gcf()
    fig.set_size_inches(5, 5)
    ax.scatter(0, 0, color="k", s=50, marker="o")
    ax.scatter(b, 0, color="k", s=50, marker="o")
    ax.scatter(p[0], -p[1], color="k", s=50, marker="o")
    ax.plot([0, b], [0, 0], color="k", lw=5)
    ax.plot([0, p[0]], [0, -p[1]], color=color, label=label)
    ax.plot([b, p[0]], [0, -p[1]], color=color, label=None)
    ax.axis("equal")
    ax.set_ylim(-1.0, None)
    return ax 

### ============= 2.a.2 Fill in below ==============

NotImplementedError

### ===========================

**Answer**: 


**2.a.3** Comment on the values of all dual variables:

- $\lambda_1$ to $\lambda_4$ for the equality constraints
- $\mu_1$ to $\mu_6$ for the bounds

What is the phyiscal meaning of these constraints? (4 pts)

Answers:


**2.a.4** Investigate the solver output (Set `print_level = 5`) and answer the following questions (8 pts)

- How many decision variables, equality constraints, and inequality constraints are present in the problem?
- What convergence criteria does IPOPT use, and how many iterations are required for convergence?
- How sensitive is the convergence to the choice of initial guesses?
- Are all constraints satisfied within the specified tolerance at convergence?

**Answer**: 


## 2.b Forward Kinematics with SQP (20 pts)

Now that you have gotten a good feel for this optimization problem and that you have a robust way of solving it, you are in a good position to try your own solvers instead of IPOPT. In class, you have seen a simple algorithm to solve equality-constrained programs: sequential quadratic programming. In what follows, we will ignore the inequality constraints from above. 

**2.b.1** Explain briefly when we can ignore the inequality constraints, and justify why it makes sense here. How can you check (a posteriori) that it was fine to ignore them? (2 pts)

**Answer**: 

**2.b.2** We will use casadi's autodiff framework to find gradients and Hessians. You could find them symbolically as well, but can be error-prone. Since the syntax is a bit confusing at first, we provide a toy example of this framework below. Using this example as guidance, implement the gradient and Hessians of the cost and constraints. Use the following unit test to check that things work: solve the problem for a given $\ell_1, \ell_2$ using IPOPT. Check the KKT conditions at the obtained solution using your newly defined function gradient and constraint Jacobian. (4 pts)

Note: IPOPT’s equality-constraint multipliers can be positive or negative. The sign depends on how the kinematic constraints are written:
For example, the constraint

$c3 = b + (l[1] + \delta[1]) \cos(\theta[1]) == p[0]$

it can be represented as either

$b + (l[1] + \delta[1]) \cos(\theta[1]) - p[0] = 0$

or equivalently

$p[0] - b - (l[1] + \delta[1]) \cos(\theta[1]) = 0.$

Both formulations define the same feasible set $g(x) = 0$, or $-g(x) = 0$, but they differ by a sign. As a result, the associated dual variable $\lambda$ will differ by a sign.


In [ ]:
# Hint: Tutorial for calculating gradient and jacobian from casadi
# ----------------------------
# Toy example:
#   f(x) = x0^2 + sin(x1)
#   g(x) = [x0 + x1,
#           x0 * x1]
# ----------------------------

# 1) Build symbolic expressions
_x = ca.SX.sym("x", 2)

_f = _x[0]**2 + ca.sin(_x[1])
_g = ca.vertcat(
    _x[0] + _x[1],
    _x[0] * _x[1],
)
_lam = ca.SX.sym("lam", _g.size1())
_L = _f + ca.dot(_lam, _g)

# 2) Build CasADi Functions (symbolic -> callable)
_f_fun    = ca.Function("f",      [_x], [_f])
_g_fun    = ca.Function("g",      [_x], [_g])
_fgradfun = ca.Function("f_grad", [_x], [ca.gradient(_f, _x)])   # shape (2,1) or (2,)
_gjacfun  = ca.Function("g_jac",  [_x], [ca.jacobian(_g, _x)])   # shape (2,2)
_L_hess   = ca.Function("L_hess", [_x, _lam], [ca.hessian(_L, _x)[0]]) # shape (2, 2)

# 3) Python-callable wrappers (NumPy in/out)
def f(x: np.ndarray) -> float:
    return float(_f_fun(x))

def g(x: np.ndarray) -> np.ndarray:
    return np.asarray(_g_fun(x), dtype=float)

def f_grad(x: np.ndarray) -> np.ndarray:
    return np.asarray(_fgradfun(x), dtype=float)

def g_jac(x: np.ndarray) -> np.ndarray:
    return np.asarray(_gjacfun(x), dtype=float)

def L_hess(x: np.ndarray, lam: np.ndarray) -> np.ndarray:
    return np.asarray(_L_hess(x, lam), dtype=float)

x0 = np.array([1.0, 0.5])  # fixed test point
l0 = np.array([0.5, 0.5])
# analytic values
print("f(x0) =", f(x0))
print("g(x0) =", g(x0))
print("f_grad(x0) =", f_grad(x0))
print("g_jac(x0) =", g_jac(x0))
print("L_hess(x0) =", L_hess(x0, l0))

In [ ]:
l = [0.5, 1.0]
l1, l2 = l

### ============= 2.b.2 Fill in below ==============



_f_fun    = None
_g_fun    = None
_fgradfun = None
_gjacfun  = None
_Lhessfun = None

### ===========================

# ---- Python-callable wrappers
def f(x: np.ndarray) -> float:
    return float(_f_fun(x))

def g(x: np.ndarray) -> np.ndarray:
    return np.asarray(_g_fun(x)).flatten()

def f_grad(x: np.ndarray) -> np.ndarray:
    return np.asarray(_fgradfun(x)).flatten()

def g_jac(x: np.ndarray) -> np.ndarray:
    return np.asarray(_gjacfun(x))
    
def L_hess(x: np.ndarray, l: np.ndarray) -> np.ndarray:
    return np.asarray(_Lhessfun(x, l), dtype=float)

def test_kkt():
    f_cost, theta, delta, p, lambdas, mus = solve_forward_kinematics(l, verbose=False)
    x0 = np.hstack([theta, delta, p])
    # Check the KKT conditions at the obtained solution using your newly defined function gradient and constraint Jacobian.
    NotImplementedError
    

test_kkt()

**2.b.2** (10 pts) Use a SQP solver to solve this problem. To make your life easier, and for you to practice "test-driven development", we have implemented a couple of unit tests for you that should pass for the developed function. 

These tests are, in order of increasing complexity: 
- test_cost: ensuring the cost at the minimum is smaller than cost at a random feasible point
- test_kkt: checking KKT conditions at the solution.
- test_ipopt: comparing solution with IPOPT

Using these tests as guidance, fill in sqp_eq_newton until all three tests pass. 

In [ ]:
def test_cost(fun, **kwargs):
    # create a random feasible point
    p0 = np.ones(2)
    t0 = np.array([np.pi / 4, 3 * np.pi / 4])
    d0 = np.array([np.sqrt(2) - l1, np.sqrt(2) - l2])

    x0 = np.hstack([t0, d0, p0])
    np.testing.assert_almost_equal(g(x0), 0.0)

    x_star, lam_star, *_ = fun(x0, **kwargs)
    assert f(x_star) <= f(x0)

def test_kkt(fun, x0, **kwargs):
    x_star, lam_star, *_ = fun(x0, **kwargs)

    Lag_x = f_grad(x_star) - g_jac(x_star).T @ (lam_star)
    np.testing.assert_almost_equal(g(x_star), 0.0)
    np.testing.assert_almost_equal(Lag_x, 0.0)

def test_ipopt(fun, x0, **kwargs):
    t0 = x0[:2]
    d0 = x0[2:4]
    p0 = x0[4:]
    f_ipopt, t_ipopt, d_ipopt, p_ipopt, lam_ipopt, mus = solve_forward_kinematics(
        l, d0=d0, t0=t0, p0=p0, verbose=False
    )
    x_ipopt = np.hstack([t_ipopt, d_ipopt, p_ipopt])

    l0 = np.full(4, 30.0)
    x_sqp, lam_sqp, info, *_ = fun(x0, l0=l0, **kwargs)

    t_sqp, d_sqp, p_sqp = x_sqp[:2], x_sqp[2:4], x_sqp[4:]
    f_sqp = f(x_sqp)

    atol = 1e-5
    assert abs(f_sqp - f_ipopt) < atol, f"not equal: {f_sqp:.5e}, {f_ipopt:.5e}"
    np.testing.assert_allclose(t_ipopt, t_sqp, atol=atol)
    np.testing.assert_allclose(d_ipopt, d_sqp, atol=atol)
    np.testing.assert_allclose(p_ipopt, p_sqp, atol=atol)
    np.testing.assert_allclose(lam_ipopt, lam_sqp, atol=atol)

In [ ]:
def sqp_eq_newton(
    x0: np.ndarray,
    l0: np.ndarray | None = None,
    max_iter: int = 50,
    tol: float = 1e-9,
    verbose: bool = False,
):
    ### ============= 2.b.2 Fill in below ==============
    
    x = np.asarray(x0, dtype=float).reshape(-1)
    m = g(x).shape[0]
    lam = l0 if l0 is not None else np.zeros(m)
    
    n = x.size
    res_pri_list = []
    res_dual_list = []
    x_list = [x.copy()]
    success = False

    alpha = 1.0

    for k in range(max_iter):
        NotImplementedError

    if verbose: 
        print("x* =", x)
        print("lambda* =", lam)
        print("g(x*) =", g(x))
    if not success: 
        print("Warning: SQP did not converge!")
    info = {"iters": k, "success": success}
    return x, lam, info, res_pri_list, res_dual_list, x_list
    ### ===========================


x0 = np.hstack([np.array([0.1, np.pi-0.1]), np.zeros(2), np.zeros(2)])  # good-ish init
l0 = np.ones(4)
x_sqp, lam_sqp, info, res_pri_list, res_dual_list, x_list = sqp_eq_newton(
    x0, l0=l0, verbose=True
)
print("info", info)
print("x* =", x_sqp)
print("lambda* =", lam_sqp)
print("g(x*) =", g(x_sqp))

test_cost(sqp_eq_newton)
test_kkt(sqp_eq_newton, x0)
test_ipopt(sqp_eq_newton, x0)

**2.b.4** Plot convergence and comment on the behavior. You can copy the code from HW2.1, 1.b.2. Compare the performance with IPOPT and comment on the difference and commonalities with IPOPT. (4 pts)

**Answer**: 

In [ ]:
### ============= 2.b.4 Fill in below ==============

def plot_convergence_rate(res_pri_list, res_dual_list, x_list):    
    NotImplementedError
    
### ===========================

plot_convergence_rate(res_pri_list, res_dual_list, x_list)

## 2.c Forward Kinematics with ALM (17 pts)

Now, let's see how the convergence changes when we use the Augmented Lagrangian with the method of multipliers instead. 

Recall that Augmented Lagrangian iterates two steps: 
- minimize (approximately) the Augmented Lagrangian, keeping current $\lambda$ fixed
- update $\lambda$ using dual ascent rule.

**2.c.1** Create the inner optimization loop which minimizes the Augmented Lagrangian for fixed $\lambda$. (5 pts)

In [ ]:
### ============= 2.c.1 Fill in below ==============
# ---- Build CasADi functions for this problem

_Lrho_fun    = None
_Lrho_grad_fun = None
_Lrho_hess_fun = None

### ===========================

def Lrho(x, lam, rho) -> np.ndarray:
    return np.asarray(_Lrho_fun(x, lam, rho), dtype=float).flatten()[0]

def Lrho_grad(x, lam, rho) -> np.ndarray:
    return np.asarray(_Lrho_grad_fun(x, lam, rho), dtype=float)
    
def Lrho_hess(x, lam, rho) -> np.ndarray:
    return np.asarray(_Lrho_hess_fun(x, lam, rho), dtype=float)

In [ ]:
### ============= 2.c.1 Fill in below ==============
# ------- Solve inner minimization
def solve_inner(x0, lam0, rho, verbose=False, max_inner=200, tol_inner=1e-2):
    alpha = 0.1
    x = x0
    lam = lam0

    success = False
    
    for i in range(max_inner):
        NotImplementedError
    
    info = {"iters": i, "success": success}
    return x, info
### ===========================

solve_inner(np.zeros(6), np.zeros(4), 0.1, verbose=False)

**2.c.2** Implement the method of multipliers method below. Verify the correctness of the solution by applying the same unit tests as above. (10 pts) 

In [ ]:
def alm_solve(
    x0,
    max_outer=200,
    max_inner=200,
    tol_outer=1e-7,
    tol_inner=1e-7,
    rho0=1e-2,
    rho_growth=10.0,
    verbose=False,
    l0 = None
):
    ### ============= 2.c.2 Fill in below ==============
    x = np.asarray(x0, float).reshape(-1)
    rho = rho0
    gx = np.asarray(g(x), float).reshape(-1)
    m = gx.size
    lam = l0 if l0 is not None else np.ones(m)

    res_pri_list = []
    res_dual_list = []
    x_list = []
    
    success = False
    alpha = 0.1

    for outer in range(max_outer):
        NotImplementedError
        
    ### ===========================
        
    info = {"success": success, "iters": outer}
    return x, lam, info, res_pri_list, res_dual_list, x_list


# ----------------------------
x0 = np.hstack([np.array([0.1, np.pi-0.1]),np.zeros(2), np.zeros(2)])
l0 = np.full(4, 30.0)

x_alm, lam_alm, info_alm, res_pri_list, res_dual_list, x_list = alm_solve(
    x0,
    verbose=True,
    l0=l0
)
print("info", info)
print("x_alm =", x_alm.round(4))
print("lambda_alm =", lam_alm)
print("g(x_alm) =", g(x_alm)) 

test_cost(alm_solve)
test_kkt(alm_solve, x0)
test_ipopt(alm_solve, x0)

In [ ]:
plot_convergence_rate(res_pri_list, res_dual_list, x_list)

**2.c.3** To conclude, briefly comment on your experience in writing these solvers. Which solver was easier to implement? Considering their different performance and ease of implementation, comment on settings in which you would prefer one solver over the other. (2 pts)

# 2.d Packaging of functions (10 pts)

**2.d.1** Move the relevant functions to the package you created in the last notebook. Then make sure you create some tests and show their outputs, as in the example below. (5 pts)

List of NLP solvers that need to move to this package:
- SQP method for NLP with equality constraints
- Augmented Lagrangian solver for NLP with equality constraints

In [ ]:
!pytest myoptim/tests

**2.d.2** Answer the questions below about the package that you created. Note that there is no right or wrong; these questions serve to make you reflect about what you have implemented. (5 pts)
- What tests did you implement? Explain each test with one short sentence.
- Are you sure that your code works based on these tests? Reflect on whether all crucial aspects of the code are tested. What could still go wrong? 
- Is the interface to your code straightforward (i.e., how many lines of code are required to run the solvers? Would it be easy for someone to use it? A good sanity check is if your code is modular so that many functions can be easily tested.) 

## Acknowledgment of Collaboration and/or Tool Use

Please choose from below (simply delete the lines that do not apply) and add a few additional notes

- “I worked alone on this assignment.”, or
- “I worked with ~~~~~~ [person or tool] on this assignment.” and/or
- “I received assistance from ~~~~~~ [person or tool] on this assignment.”

For the last two cases, specify how the person or tool helped you and explain why this amplified your learning process:

_add answer here_